In [ ]:
import pandas as pd
import numpy as np

from sklearn.impute import KNNImputer

# Graphs
import matplotlib.pyplot as plt

# Preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from imblearn.over_sampling import SMOTE

# Metrics
from sklearn.metrics import roc_curve, roc_auc_score, auc, ConfusionMatrixDisplay, classification_report
from sklearn import tree

# SHAP values
import shap

In [ ]:
def data_prep(df, topogrup):
    """
    Prepara o conjunto de dados para análise, aplicando filtros e transformações necessárias.

    Parâmetros:
    -----------
    df : pandas.DataFrame
        O conjunto de dados que será preparado.
    topogrup : list
        Lista com os códigos do grupo da topografia para serem selecionados.

    Retorno:
    --------
    pandas.DataFrame
        O conjunto de dados após as transformações.

    """

    df_aux = df.copy()

    # TOPOGRUP
    df_aux = df_aux[df_aux.TOPOGRUP.isin(topogrup)]

    # IDADE > 19
    df_aux = df_aux[df_aux.IDADE > 19]

    # Somente residentes do estado de SP
    df_aux = df_aux[df_aux.UFRESID == 'SP']

    # ECGRUP sem 0, X e Y
    df_aux = df_aux[~df_aux.ECGRUP.isin(['0','X','Y'])]

    # ANODIAG até 2019 e em categorias de 5 em 5 anos
    df_aux = df_aux[df_aux.ANODIAG < 2020]
    # df_aux.ANODIAG = [0 if x < 2005 else 1 if x < 2010 else 2 if x < 2015 else 3 for x in df_aux.ANODIAG]

    # Somente com confirmação microscópica
    df_aux = df_aux[df_aux.BASEDIAG == 3]

    # Somente MORFO 81403
    # df_aux = df_aux[df_aux.MORFO == 81403]

    # Tratamento das colunas com datas
    list_datas = ['DTCONSULT', 'DTDIAG', 'DTTRAT', 'DTULTINFO']

    for col_data in list_datas:
        df_aux[col_data] = pd.to_datetime(df_aux[col_data])

    df_aux['CONSDIAG'] = (df_aux.DTDIAG - df_aux.DTCONSULT).dt.days
    df_aux['DIAGTRAT'] = (df_aux.DTTRAT - df_aux.DTDIAG).dt.days
    df_aux['TRATCONS'] = (df_aux.DTTRAT - df_aux.DTCONSULT).dt.days
    df_aux['ULTIDIAG'] = (df_aux.DTULTINFO - df_aux.DTDIAG).dt.days
    df_aux[['DIAGTRAT', 'TRATCONS']] = df_aux[['DIAGTRAT', 'TRATCONS']].fillna(-1)

    df_aux.CONSDIAG = [0 if consdiag <= 30 else 1 if consdiag <= 60 else 2 for consdiag in df_aux.CONSDIAG]
    df_aux.TRATCONS = [99 if tratcons < 0 else 0 if tratcons <= 60 else 1 if tratcons <= 90 else 2 for tratcons in df_aux.TRATCONS]
    df_aux.DIAGTRAT = [99 if diagtrat < 0 else 0 if diagtrat <= 60 else 1 if diagtrat <= 90 else 2 for diagtrat in df_aux.DIAGTRAT]

    # Input da ESCOLARI usando ECGRUP, IDADE e SEXO
    cols_input = ['ESCOLARI', 'IDADE', 'ECGRUP', 'SEXO']

    df_esc = df_aux[cols_input].copy()
    df_esc.loc[df_esc.ESCOLARI == 9, 'ESCOLARI'] = np.nan
    df_esc.loc[df_esc.ECGRUP == 'I', 'ECGRUP'] = 1
    df_esc.loc[df_esc.ECGRUP == 'II', 'ECGRUP'] = 2
    df_esc.loc[df_esc.ECGRUP == 'III', 'ECGRUP'] = 3
    df_esc.loc[df_esc.ECGRUP == 'IV', 'ECGRUP'] = 4

    X = df_esc.values
    imputer = KNNImputer(n_neighbors=5, weights='distance')
    df_aux['ESCOLARI_preench'] = imputer.fit_transform(X)[:,0].round()

    # Variável para pessoas que fazem tratamento na mesma cidade (0 = Não, 1 = Sim)
    df_aux['IBGE_idem_IBGEATEN'] = 0
    df_aux.loc[df_aux.IBGE == df_aux.IBGEATEN, 'IBGE_idem_IBGEATEN'] = 1

    # Variável para presença de recidiva (0 = Não, 1 = Sim)
    df_aux['presenca_rec'] = [0 if rec == 1 else 1 for rec in df_aux.RECNENHUM]
    df_aux.loc[(df_aux.presenca_rec == 0) & (df_aux.DTRECIDIVA.notnull()), 'presenca_rec'] = 1

    # Somente o número da DRS
    DRS_expand = df_aux.DRS.str.split(' ', expand=True)
    df_aux['DRS'] = DRS_expand[1]

    # Retirada das colunas que não serão usadas
    cols = df_aux.columns
    drop_cols = ['ESCOLARI', 'UFNASC', 'UFRESID', 'CIDADE', 'DTCONSULT', 'CLINICA',
                 'DTDIAG', 'BASEDIAG', 'DESCTOPO', 'MORFO', 'DESCMORFO', 'ECGRUP',
                 'T', 'N', 'M', 'PT', 'PN', 'PM', 'S', 'G', 'LOCALTNM', 'IDMITOTIC',
                 'PSA', 'GLEASON', 'OUTRACLA', 'META01', 'META02', 'META03',
                 'META04', 'DTTRAT', 'NAOTRAT', 'TRATAMENTO', 'TRATFANTES',
                 'TRATFAPOS', 'NENHUMANT', 'CIRURANT', 'RADIOANT', 'QUIMIOANT',
                 'HORMOANT', 'TMOANT', 'IMUNOANT', 'OUTROANT', 'NENHUMAPOS',
                 'CIRURAPOS', 'RADIOAPOS', 'QUIMIOAPOS', 'HORMOAPOS', 'TMOAPOS',
                 'IMUNOAPOS', 'OUTROAPOS', 'DTULTINFO', 'CICI', 'CICIGRUP',
                 'CICISUBGRU', 'FAIXAETAR', 'LATERALI', 'INSTORIG', 'PERDASEG',
                 'ERRO', 'DTRECIDIVA', 'RECNENHUM', 'RECLOCAL', 'RECREGIO',
                 'RECDIST', 'REC01', 'REC02', 'REC03', 'REC04', 'CIDO', 'DSCCIDO',
                 'HABILIT', 'HABIT11', 'HABILIT1', 'CIDADEH']

    cols = cols.drop(drop_cols)

    return df_aux[cols]

#######################################################################################

def get_labels(df):
    """
    Cria novas colunas para indicar se o paciente morreu, se morreu devido a câncer e se está vivo após 1, 3 e 5 anos
    a partir do diagnóstico.

    Parâmetros:
    -----------
    df : pandas.DataFrame
        DataFrame contendo os dados dos pacientes.

    Retorno:
    --------
    pandas.DataFrame
        DataFrame com as novas colunas adicionadas.

    """

    df_aux = df.copy()

    df_aux['obito_geral'] = 0
    # df_aux['obito_cancer'] = 0

    df_aux['sobrevida_ano1'] = 0
    df_aux['sobrevida_ano3'] = 0
    df_aux['sobrevida_ano5'] = 0

    df_aux.loc[df_aux.ULTINFO > 2, 'obito_geral'] = 1

    # df_aux.loc[df_aux.ULTINFO == 3, 'obito_cancer'] = 1

    df_aux.loc[df_aux.ULTIDIAG > 365, 'sobrevida_ano1'] = 1
    df_aux.loc[df_aux.ULTIDIAG > 3*365, 'sobrevida_ano3'] = 1
    df_aux.loc[df_aux.ULTIDIAG > 5*365, 'sobrevida_ano5'] = 1

    # df_ano1 = df_aux[~((df_aux.obito_geral == 0) & (df_aux.sobrevida_ano1 == 0))].reset_index(drop=True)

    drop = ['ULTINFO', 'ULTIDIAG']#'obito_geral']

    return df_aux.drop(columns=drop)

#######################################################################################

def get_train_test(df, drop_cols, label, test_size=0.25, random_state=0):
    """
    Divide o dataframe em dados de treino e teste, de acordo com uma variável target e uma lista de colunas a serem
    excluídas. Retorna quatro dataframes: X_train, X_test, y_train, y_test.

    Parâmetros:
    -----------
    df : DataFrame
        Dataframe que será dividido em treino e teste.
    drop_cols : list
        Lista de colunas a serem excluídas do dataframe.
    label : str
        Nome da coluna que é a variável target.
    test_size : float, opcional (padrão=0.25)
        Tamanho da fração dos dados que será utilizada para teste.
    random_state : int, opcional (padrão=0)
        Semente para a geração de números aleatórios.

    Retorno:
    --------
    X_train : DataFrame
        Dataframe de treino com as features.
    X_test : DataFrame
        Dataframe de teste com as features.
    y_train : DataFrame
        Dataframe de treino com a variável target.
    y_test : DataFrame
        Dataframe de teste com a variável target.

    """

    df_aux = df.copy()

    cols = df_aux.columns.drop(drop_cols)
    lb = df_aux[label].copy()
    cols = cols.drop(label)
    feat = df_aux[cols]

    X_train, X_test, y_train, y_test = train_test_split(feat, lb,
                                                        test_size=test_size,
                                                        random_state=random_state,
                                                        stratify=lb)

    return X_train, X_test, y_train, y_test

#######################################################################################

def train_preprocessing(df, ohe_encoder=None, normalizer='StandardScaler'):
    """
    Realiza o pré-processamento do conjunto de treino.

    Parâmetros:
    -----------
    df : pandas.DataFrame
        O conjunto de dados a ser pré-processado.
    ohe_encoder : lista, opcional
        Lista com as colunas para realizar o One Hot Encoder, se None, todas as colunas serão codificadas com o Label Encoder.
    normalizer : str, opcional
        Tipo de normalização a ser usada para escalonamento das variáveis (padrão é 'StandardScaler').

    Retorno:
    --------
    df_aux : pandas.DataFrame
        O conjunto de dados pré-processado.
    enc : dict
        Dicionário de codificadores de rótulos, contendo um codificador para cada variável categórica.
    norm : objeto
        Normalizador de dados utilizado para escalonamento das variáveis.
    feat_cols : lista
        Lista com o nome das colunas das variáveis do modelo.

    """
    df_aux = df.copy()

    enc = dict()
    if ohe_encoder != None:
        for col in ohe_encoder:
            enc[col] = OneHotEncoder(handle_unknown='ignore', drop='first')
            ohe_results = enc[col].fit_transform(df_aux[[col]])
            df1 = pd.DataFrame(ohe_results.toarray(),
                               columns=enc[col].get_feature_names_out(),
                               index=df_aux[col].index)
            df_aux = df_aux.merge(df1, how='left', left_index=True, right_index=True)

        df_aux.drop(columns=ohe_encoder, inplace=True)

    list_categorical = df_aux.select_dtypes(include='object').columns
    for col in list_categorical:
        enc[col] = LabelEncoder()
        df_aux[col] = enc[col].fit_transform(df_aux[col])

    feat_cols = df_aux.columns

    if normalizer == 'StandardScaler':
        norm = StandardScaler()
    elif normalizer == 'MinMaxScaler':
        norm = MinMaxScaler((0, 1))
    elif normalizer == 'MaxAbsScaler':
        norm = MaxAbsScaler()
    elif normalizer == 'QuantileTransformer':
        norm = QuantileTransformer(output_distribution='normal')

    df_aux = norm.fit_transform(df_aux)

    return df_aux, enc, norm, feat_cols

#######################################################################################

def test_preprocessing(df, enc, norm, ohe_encoder=None):
    """
    Realiza o pré-processamento do conjunto de teste.

    Parâmetros:
    -----------
    df : pandas DataFrame
        O conjunto de dados a ser pré-processado.
    enc : dict
        Dicionário contendo os objetos encoders utilizados para pré-processar os dados de treino.
    norm : objeto de normalização
        Objeto de normalização utilizado para pré-processar os dados de treino.
    ohe_encoder : lista, padrão None
        Lista com as colunas para realizar o One Hot Encoder, se None, todas as colunas serão codificadas com o Label Encoder.

    Retorno:
    --------
    pandas DataFrame
        O conjunto de dados pré-processado.

    """

    df_aux = df.copy()

    if ohe_encoder != None:
        for col in ohe_encoder:
            ohe_results = enc[col].transform(df_aux[[col]])
            df1 = pd.DataFrame(ohe_results.toarray(),
                               columns=enc[col].get_feature_names_out(),
                               index=df_aux[col].index)
            df_aux = df_aux.merge(df1, how='left', left_index=True, right_index=True)

        df_aux.drop(columns=ohe_encoder, inplace=True)

    list_categorical = df_aux.select_dtypes(include='object').columns
    for col in list_categorical:
        df_aux.loc[~df_aux[col].isin(enc[col].classes_), col] = -1
        df_aux.loc[df_aux[col].isin(enc[col].classes_), col] = enc[col].transform(df_aux[col][df_aux[col].isin(enc[col].classes_)])

    df_aux = norm.transform(df_aux)

    return df_aux

#######################################################################################

def preprocessing(df, cols_drop, label, test_size=0.25, ohe_encoder=None,
                  norm_name='StandardScaler', return_enc_norm=False,
                  balance_data=False, random_state=0):
    """
    Realiza o pré-processamento dos dados de treino e teste.

    Parâmetros:
    -----------
    df : pandas DataFrame
        O DataFrame contendo as features e o label.
    cols_drop : lista
        Lista com o nome das colunas que devem ser removidas do DataFrame.
    label : str
        O nome da coluna do label.
    test_size : float, padrão 0.25
        Tamanho da porcentagem dos dados que serão usados para teste.
    ohe_encoder : lista, padrão None
        Lista com as colunas para realizar o One Hot Encoder, se None, todas as colunas serão codificadas com o Label Encoder.
    norm_name : str, padrão 'StandardScaler'
        Nome do normalizador a ser usado, podendo ser 'StandardScaler', 'MinMaxScaler', 'MaxAbsScaler' ou 'QuantileTransformer'.
    return_enc_norm : bool, padrão False
        Se True, retorna os objetos enc e norm que foram usados no pré-processamento dos dados.
    balance_data : bool, padrão False
        Se True, realiza balanceamento dos dados usando a técnica SMOTE.
    random_state : int, padrão 0
        Semente aleatória para reprodutibilidade.

    Retornos:
    ---------
    X_train_ : numpy ndarray
        Matriz de features de treino pré-processada.
    X_test_ : numpy ndarray
        Matriz de features de teste pré-processada.
    y_train_ : numpy ndarray
        Array da variável alvo de treino.
    y_test : numpy ndarray
        Array da variável alvo de teste.
    feat_cols : lista
        Lista com o nome das colunas das features após o pré-processamento.
    enc : dict
        Dicionário contendo os objetos LabelEncoder ou OneHotEncoder de cada coluna categórica.
        (somente se return_enc_norm for True)
    norm : objeto normalizador
        Objeto normalizador de cada coluna numérica.
        (somente se return_enc_norm for True)

    """
    df_aux = df.copy()

    # Train Test split
    X_train, X_test, y_train, y_test = get_train_test(df_aux, cols_drop, label,
                                                      test_size,
                                                      random_state=random_state)

    # Preprocessing
    X_train_enc, enc, norm, feat_cols = train_preprocessing(X_train, ohe_encoder=ohe_encoder,
                                                            normalizer=norm_name)
    X_test_ = test_preprocessing(X_test, enc, norm, ohe_encoder=ohe_encoder)

    # Balancing
    if balance_data:
        X_train_, y_train_ = SMOTE(random_state=random_state).fit_resample(X_train_enc, y_train)

    else:
        X_train_, y_train_ = X_train_enc, y_train

    print(f'X_train = {X_train_.shape}, X_test = {X_test_.shape}')
    print(f'y_train = {y_train_.shape}, y_test = {y_test.shape}')

    if return_enc_norm:
        return X_train_, X_test_, y_train_, y_test, feat_cols, enc, norm
    else:
        return X_train_, X_test_, y_train_, y_test, feat_cols

#######################################################################################

def show_tree(model, feat_cols, max_depth=3, estimator=0):
    """
    Plota a árvore de decisão de um modelo RandomForestClassifier.

    Parâmetros:
    -----------
    model : RandomForestClassifier
        Modelo treinado do tipo RandomForestClassifier.
    feat_cols : array-like
        Lista ou array com os nomes das features utilizadas no treinamento.
    max_depth : int, opcional
        A profundidade máxima da árvore a ser exibida. O valor padrão é 3.
    estimator : int, opcional
        O índice do estimador a ser exibido. O valor padrão é 0.

    Retorno:
    --------
    None

    """
    plt.figure(figsize = (22, 10))
    tree.plot_tree(model.estimators_[estimator],
                   feature_names=feat_cols,
                   filled=True,
                   max_depth=max_depth);

#######################################################################################

def plot_feat_importances(model, feat_cols, n=10):
    """
    Plota o gráfico de barras com as n características mais importantes de um modelo.

    Parâmetros:
    -----------
    model: objeto
        Objeto do modelo já treinado.

    feat_cols: array-like
        Lista ou array contendo o nome das colunas das features.

    n: int, default=10
        Número de features mais importantes a serem plotadas.

    Retorno:
    --------
    None

    """

    feat_import = pd.Series(model.feature_importances_, index=feat_cols)
    feat_import.nlargest(n).plot(kind='barh', figsize=(10, 8))
    plt.show()

#######################################################################################

def plot_roc_curve(model, X_train, X_test, y_train, y_test):
    """
    Plota a curva ROC para o modelo especificado usando as previsões de probabilidade de treino e teste.
    AUC é calculado para as curvas de treino e teste e plotado no gráfico.

    Parâmetros:
    -----------
    model : modelo de classificação
        Um modelo de classificação treinado com método `fit` e que tenha um método `predict_proba`.
    X_train : pandas DataFrame
        Dados de treino contendo as features.
    X_test : pandas DataFrame
        Dados de teste contendo as features.
    y_train : pandas Series
        Variável label correspondente aos dados de treino.
    y_test : pandas Series
        Variável label correspondente aos dados de teste.

    Retorno:
    --------
    None

    """
    probas_train = model.predict_proba(X_train)[:, 1]
    probas_test = model.predict_proba(X_test)[:, 1]

    fp_train, tp_train, _ = roc_curve(y_train, probas_train)
    fp_test, tp_test, _ = roc_curve(y_test, probas_test)

    plt.figure(figsize=(10, 7))
    plt.plot(fp_train, tp_train, 'b', label=f'Train (AUC = {auc(fp_train, tp_train):.4f})')
    plt.plot(fp_test, tp_test, 'r', label=f'Test (AUC = {auc(fp_test, tp_test):.4f})')
    plt.plot(np.linspace(0, 1, 100),
             np.linspace(0, 1, 100),
             label='Baseline',
             linestyle='--',
             color='k')
    plt.xlabel('False Positives')
    plt.ylabel('True Positives')
    plt.grid(True)
    plt.legend()
    plt.show()

#######################################################################################

def plot_confusion_matrix(model, x, y, format='.4f', norm=True):
    """
    Plota uma matriz de confusão normalizada e um relatório de classificação.

    Parâmetros:
    -----------
    model : estimador
        Um modelo de classificação já treinado.
    x : array-like, shape (n_samples, n_features)
        Conjunto de teste com as features.
    y : array-like, shape (n_samples,)
        Conjunto de teste com o label.
    format : str, opcional (padrão='.3f')
        O formato a ser usado para os valores na matriz de confusão.

    Retorno:
    --------
    None

    """
    with plt.rc_context({'font.size': 12, 'font.weight': 'bold'}):
        if norm:
            ConfusionMatrixDisplay.from_estimator(model, x, y, values_format=format,
                                                  cmap='binary', normalize='true')
        else:
            ConfusionMatrixDisplay.from_estimator(model, x, y, cmap='binary',
                                                  values_format='.1f')
        plt.show()

    print(f'\n{classification_report(y, model.predict(x), digits=4)}')

#######################################################################################

def plot_shap_values(model, x, features, max_display=10):
    """
    Plota o gráfico de valores shapley para cada feature.

    Parâmetros:
    -----------
    model: modelo treinado
        Modelo que será utilizado para calcular os valores shapley.
    x: pd.DataFrame
        Dados de entrada que serão utilizados para calcular os valores shapley.
    features: list
        Lista de strings com o nome das colunas das features.
    max_display: int, optional
        Número máximo de features a serem exibidas no gráfico. Padrão é 10.

    Retorno:
    --------
    None

    """
    shap_values = shap.TreeExplainer(model).shap_values(x)

    try:
        shap.summary_plot(shap_values[1], x,
                          feature_names=features,
                          max_display=max_display)
    except AssertionError:
        shap.summary_plot(shap_values, x,
                          feature_names=features,
                          max_display=max_display)

#######################################################################################

def roc_together(X, y, naive_bayes=None, random_forest=None, xgboost=None,
                 lightgbm=None):
    """
    Plota a curva ROC para vários modelos.

    Parâmetros:
    -----------
    X: array-like
        Array com os valores das features.
    y: array-like
        Array com os valores do target.
    naive_bayes: objeto da classe GaussianNB, opcional (padrão=None)
        Objeto do modelo Gaussian Naive Bayes.
    random_forest: objeto da classe RandomForestClassifier, opcional (padrão=None)
        Objeto do modelo Random Forest Classifier.
    xgboost: objeto da classe XGBClassifier, opcional (padrão=None)
        Objeto do modelo XGBoost Classifier.
    lightgbm: objeto da classe LGBMClassifier, opcional (padrão=None)
        Objeto do modelo LightGBM Classifier.

    Retorno:
    --------
    None

    """
    plt.figure(figsize=(10, 7))

    if naive_bayes != None:
        probas_nb = naive_bayes.predict_proba(X)[:, 1]
        fp_nb, tp_nb, _ = roc_curve(y, probas_nb)
        plt.plot(fp_nb, tp_nb, 'k', linestyle='dashed',
                 label=f'Naive Bayes (AUC = {auc(fp_nb, tp_nb):.4f})')

    if random_forest != None:
        probas_rf = random_forest.predict_proba(X)[:, 1]
        fp_rf, tp_rf, _ = roc_curve(y, probas_rf)
        plt.plot(fp_rf, tp_rf, 'k', linestyle='dashdot',
                 label=f'Random Forest (AUC = {auc(fp_rf, tp_rf):.4f})')

    if xgboost != None:
        probas_xgb = xgboost.predict_proba(X)[:, 1]
        fp_xgb, tp_xgb, _ = roc_curve(y, probas_xgb)
        plt.plot(fp_xgb, tp_xgb, 'k',
                 label=f'XGBoost (AUC = {auc(fp_xgb, tp_xgb):.4f})')

    if lightgbm != None:
        probas_lgbm = lightgbm.predict_proba(X)[:, 1]
        fp_lgbm, tp_lgbm, _ = roc_curve(y, probas_lgbm)
        plt.plot(fp_lgbm, tp_lgbm, 'k', linestyle='dashed',
                 label=f'LightGBM (AUC = {auc(fp_lgbm, tp_lgbm):.4f})')

    plt.plot(np.linspace(0, 1, 100),
             np.linspace(0, 1, 100),
             label='Baseline',
             linestyle='dotted',
             color='gray')
    plt.xlabel('False Positives')
    plt.ylabel('True Positives')
    plt.grid(True)
    plt.legend()
    plt.show()

#######################################################################################

def pred_cruzada(df, model, list_drop, label, enc, norm):

    # Filtro para os anos de sobrevida de interesse
    df_sobrevida = df[~((df.obito_geral == 0) & (df[label] == 0))].reset_index(drop=True)

    # Saída
    y = df_sobrevida[label].values

    # Entradas
    X = df_sobrevida.drop(columns=list_drop)
    X_pp = test_preprocessing(X, enc, norm)

    # Matriz de confusão
    plot_confusion_matrix(model, X_pp, y)